# Applied Data Science Capstone: Week 3 
## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada, Part 2

A Jupyter Notebook that uses pandas and other python libraries to demonstrate 
*k means clustering*. Builds on work in Part 1, which is a separate notebook.
The new work begins under the "Part 2 code starts here" heading.

## Code from Part 1

I've condensed all the code from the Part 1 notebook into a smaller group of cells.

In [16]:
import pandas as pd
import lxml

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
raw_postal_codes = df[0]
raw_postal_codes = raw_postal_codes[(raw_postal_codes.Borough != 'Not assigned')]
raw_postal_codes = raw_postal_codes[(raw_postal_codes.Neighbourhood != 'Not assigned')]

grouped = raw_postal_codes.groupby('Postal Code')
grouped_data = {'PostalCode':[], 'Borough':[], 'Neighborhood':[]}
for a, b in grouped:
    grouped_data['PostalCode'].append(a)
    grouped_data['Borough'].append(', '.join(b['Borough'].tolist()))
    grouped_data['Neighborhood'].append(', '.join(b['Neighbourhood'].tolist()))

postal_codes = pd.DataFrame(grouped_data)
print("Dataframe ready!")
print(postal_codes.shape)

Dataframe ready!
(103, 3)


## Part 2 code starts here

Start with the geocoder code from Coursera. 

In [20]:
pip install geocoder

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.7/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
import geocoder

# initialize your variable to None

# Small change from Coursera - as a matter of
# style, I don't like to let a loop run forever
# until is succeeds, so I added a counter. Good
# thing I did: the geocode code never returned 
# a value for me, no matter how many times I
# let the loop run

lat_lng_coords = None
found = 0
not_found = 0

for p in postal_codes['PostalCode']:
    print(".", end='')
    i = 0
    while(lat_lng_coords is None and i <= 10):
        i += 1
        g = geocoder.google("{}, Toronto, Ontario, Canada".format(p))
        lat_lng_coords = g.latlng
    if lat_lng_coords is None:
        not_found += 1
    else:
        found += 1

print("\nfound: {}\nnot found: {}".format(found, not_found))

.......................................................................................................
found: 0
not found: 103


## Another attempt using geopy

The New York notebook used geopy, so I thought
to give it a try here.

In [23]:
pip install geopy

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.7/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from geopy.geocoders import Nominatim

found = 0
not_found = 0

geolocator = Nominatim(user_agent="Capstone Week 3")
for i in postal_codes['PostalCode']:
    print(".", end='')
    location = geolocator.geocode("{}, Toronto, Ontario, Canada".format(i))
    if location is not None:
        found += 1
    else:
        not_found += 1

print("\nfound: {}\nnot found: {}".format(found, not_found))

NameError: name 'postal_codes' is not defined

## Using the Coursera CSV File

I tried. I think if I were doing this in a production environment,
I would get a Google developer API key and use their service.

In [25]:
long_lat = pd.read_csv("https://cocl.us/Geospatial_data")
long_lat.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
postal_codes = postal_codes.merge(long_lat, on='PostalCode')
postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
postal_codes.shape

(103, 5)